In [12]:
%pip install altair pandas numpy plotly
import altair as alt 
alt.data_transformers.enable('json')

Note: you may need to restart the kernel to use updated packages.


DataTransformerRegistry.enable('json')

In [13]:
# imports
import pandas as pd
# import matplotlib.pyplot as plt
import plotly.graph_objs as go
import numpy as np

In [14]:
df = pd.read_csv("data/dpt2020.csv", sep=";")

# Vérifier si '2A' ou '2B' sont présents
print("'2A' présent ?", '2A' in df['dpt'].unique())
print("'2B' présent ?", '2B' in df['dpt'].unique())

'2A' présent ? False
'2B' présent ? False


In [15]:
print(f"Initial shape is {df.shape}")

# supp years and dpt = XXXX and transform to int
to_clean=['annais', 'dpt']
for col in to_clean:
    #df[col] = df[col].astype(str).str.strip()
    
    # Supprimer les lignes où la colonne n'est pas strictement numérique
    df = df[df[col].str.isnumeric()]
    
    if col == 'annais':
        df[col] = df[col].astype(int)
        
    else:
    # Reforcer le type string natif (facultatif mais propre)
        df[col] = df[col].astype("string")
        
        ### Ce n'est pas supported par la carte que nous avons ###
        # Transformation pour la corse => tout est transformé en 2A
        df.loc[df['dpt'] == '20', 'dpt'] = '2A'
        # Exclusion des DOM_TOM
        df = df[~df['dpt'].isin(['971', '972', '973', '974'])]
    print(df[col].dtype)
    print(f"Shape is now {df.shape}")

Initial shape is (3727553, 5)
int64
Shape is now (3690309, 5)
string
Shape is now (3492154, 5)


In [16]:
# Supprimer les prénoms "_PRENOMS_RARES"
df = df[df["preusuel"] != "_PRENOMS_RARES"]
# Ajouter une colonne de longueur de prénom
df["longueur"] = df["preusuel"].str.len()
# Supprimer les prénoms de longueur 1
df = df[df["longueur"] > 1]
print(f"Shape is now {df.shape}")

Shape is now (3471072, 6)


In [17]:
df

,sexe,preusuel,annais,dpt,nombre,longueur
10885,1,AADIL,1983,84,3,5
10886,1,AADIL,1992,92,3,5
10888,1,AAHIL,2016,95,3,5
10892,1,AARON,1962,75,3,5
10893,1,AARON,1976,75,3,5
...,...,...,...,...,...,...
3727543,2,ZYA,2011,91,3,3
3727545,2,ZYA,2013,44,4,3
3727546,2,ZYA,2013,59,3,3
3727548,2,ZYA,2018,59,3,3


In [18]:
df['fin'] = df['preusuel'].str[-1].str.lower()
# On prépare un DataFrame pour le camembert avec toutes les terminaisons
df_pie = df.groupby(['annais', 'fin'])['nombre'].sum().reset_index()

In [19]:
print(sorted(df['fin'].unique()))

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'â', 'ç', 'é', 'ë', 'ï']


In [20]:
def top10_terminaisons(df_pie, annee):
    # Filtrer pour l'année sélectionnée
    data_annee = df_pie[df_pie['annais'] == annee].copy()
    # Trier et sélectionner les 5 plus fréquentes
    top5 = data_annee.nlargest(10, 'nombre')
    autres = data_annee[~data_annee['fin'].isin(top5['fin'])]
    # Additionner les "autres"
    autres_sum = autres['nombre'].sum()
    # Créer la ligne "autres" si besoin
    if autres_sum > 0:
        autres_row = pd.DataFrame([{
            'annais': annee,
            'fin': 'autres',
            'nombre': autres_sum
        }])
        top10 = pd.concat([top5, autres_row], ignore_index=True)
    return top10

In [21]:
annee_slider = alt.binding_range(
    min=df_pie['annais'].min(),
    max=df_pie['annais'].max(),
    step=1,
    name="Année :"
)
annee_param = alt.param(
    name="année",
    bind=annee_slider,
    value=df_pie['annais'].min()
)

base = alt.Chart(df_pie).transform_filter(
    alt.datum.annais == annee_param
).transform_window(
    rank='rank(nombre)',
    sort=[alt.SortField('nombre', order='descending')]
).transform_calculate(
    fin_affiche="datum.rank <= 10 ? datum.fin : 'autres'"
).transform_aggregate(
    nombre='sum(nombre)',
    groupby=['fin_affiche']
).transform_window(
    order_rank='rank(-datum.nombre)',
    sort=[alt.SortField('nombre', order='descending')]
).mark_arc().encode(
    theta=alt.Theta("nombre:Q", stack=True),
    color=alt.Color(
        "fin_affiche:N",
        title="Terminaison",
        sort=alt.EncodingSortField(field="nombre", op="sum", order="descending")
    ),
    tooltip=["fin_affiche:N", "nombre:Q"]
).add_params(
    annee_param
).properties(
    title="Répartition des prénoms selon leur terminaison (top 10, autres regroupés)"
)

base

alt.Chart(...)